# Preparation

In [ ]:
import pandas as pd
import numpy as np
import pickle
import configparser
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.style.use('ggplot')

from tqdm import tqdm_notebook as tqdm
from glob import glob
pd.set_option('display.max_columns', None)

import pandas as pd
import statsmodels.formula.api as smf 
import statsmodels.api as sm  
import seaborn as sns
import matplotlib.pyplot as plt 
pd.set_option('display.max_columns', None)


%matplotlib inline

username= ## insert your username ##
password= ## insert your password ##
database= ## insert your databasename ##
appid = ## insert your appid for KAKEN ##
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'
engine = create_engine(url)

# Category C

## Data Importing + Cleaning DF_C_A1

SQL REQUIREMENTS:
- Duration: 2004-2017
- Category: C (niicode == 72)
- Section: 17 (general research)
- Project Status: closed (completed)
- Project Type: project

In [ ]:
sql = '''
SELECT *
FROM grantaward
WHERE grantaward.startfiscalyear BETWEEN 2004 AND 2017
AND grantaward.category_niicode = 72
AND grantaward.section_niicode = 17
AND grantaward.projectstatus_statuscode = "project_closed"
AND grantaward.projecttype = "project"
'''

# df : the original dataframe stated above
df = pd.read_sql_query(sql, engine)
df = df.set_index("awardnumber")

In [ ]:
df = df.drop(['projecttype','projectstatus_fiscalyear', 'projectstatus_statuscode','category_niicode',
       'section_niicode','institution_mextcode',
       'institution_jspscode', 'sequence', 'participate',
       'eradcode','familyname_yomi','givenname_yomi'], axis=1)
df['project_length'] = df['endfiscalyear'] - df['startfiscalyear'] + 1
df = df.replace({'kikin': 1, 'hojokin': 0})
options = [3,4,5]
df = df[df["project_length"].isin(options)]
df = df.sort_values(by = 'startfiscalyear')
df = df.drop('23593141')
df.head(2) #110111 rows × 12 columns


In [ ]:
#pivot = pd.crosstab(df['startfiscalyear'], df['allocation_niicode'], margins = True)
#pivot

In [ ]:
sql_saimoku = '''
SELECT * 
FROM grantaward_field
'''
df_saimoku = pd.read_sql_query(sql_saimoku, engine)
df_saimoku.head()

In [ ]:
sql_institutiontype = '''
SELECT *
FROM master_institutions
'''
df_institution_type = pd.read_sql_query(sql_institutiontype, engine)
df_institution_type = df_institution_type[['institution_niicode', 'institution_table_group']]
df_institution_type


### Saimoku list
- 556 -> 939: Integraded Disciplines
- 621: Humanities and Social sciences
- 682: Sciences and Engineering
- 770 Biological Sciences

In [ ]:
options = [621, 682, 770, 556, 939]
df_saimoku_all = df_saimoku[df_saimoku['field_niicode'].isin(options)]
df_saimoku_all = df_saimoku_all.drop(['index', 'field_sequence', 'field_path','field_table','field_name'], axis=1)
df_saimoku_all.head() #325476 rows × 2 columns

In [ ]:
df = df.reset_index()
df.head(2)

In [ ]:
df_limited_saimoku = df.merge(df_saimoku_all,  how='inner', on = 'awardnumber')
df_limited_saimoku.head(2)

df_limited_saimoku = df_limited_saimoku.replace({556: 'INT', 682: 'SE',939: 'INT', 770: 'BIO',621:'HS'})
df_limited_saimoku.head(2)

In [ ]:
df_limited_saimoku_institution = df_limited_saimoku.merge(df_institution_type,  how='inner', on = 'institution_niicode')

In [ ]:
df_limited_saimoku_institution.shape

### descriptive statistisc

In [ ]:
df_HS = df_limited_saimoku_institution[df_limited_saimoku_institution['field_niicode'] == 'HS']
df_SE = df_limited_saimoku_institution[df_limited_saimoku_institution['field_niicode'] == 'SE']
df_BIO = df_limited_saimoku_institution[df_limited_saimoku_institution['field_niicode'] == 'BIO']
df_INT = df_limited_saimoku_institution[(df_limited_saimoku_institution['field_niicode'] == 'INT')]

In [ ]:
df_checking = df_limited_saimoku_institution###

print("______________")
print("NUMBER OF PROJECTS")

pivot = pd.crosstab(df_checking['startfiscalyear'], df_checking['allocation_niicode'], margins = True)
display(pivot)
print("______________")
print("INSTITUTION TYPE")


pivot = pd.crosstab(df_checking['allocation_niicode'], df_checking['institution_table_group'], margins = True, margins_name='Total')
display(pivot)

pivot = pd.crosstab(df_checking['startfiscalyear'], df_checking['institution_table_group'], margins = True, margins_name='Total')
display(pivot)


pivot = pd.crosstab(df_checking['allocation_niicode'], df_checking['institution_table_group'], margins = True, margins_name='Total')
pivot

# Define a function to normalize each row
def row_normalize(row):
    row_sum = row['Total']
    return row.iloc[:-1] / row_sum

# Apply the normalization function to each row
normalized_pivot = pivot.apply(row_normalize, axis=1)

# Print the normalized crosstab
display(normalized_pivot)

pivot = pd.crosstab(df_checking['startfiscalyear'], df_checking['institution_table_group'], margins = True, margins_name='Total')
pivot

# Define a function to normalize each row
def row_normalize(row):
    row_sum = row['Total']
    return row.iloc[:-1] / row_sum

# Apply the normalization function to each row
normalized_pivot = pivot.apply(row_normalize, axis=1)

# Print the normalized crosstab
display(normalized_pivot)

print("______________")
print("DIRECT COST")

grouped = df_checking.groupby('startfiscalyear')
stats_by_directcost = grouped['directcost'].describe()
display(stats_by_directcost)

grouped = df_checking.groupby('allocation_niicode')
stats_by_directcost = grouped['directcost'].describe()
display(stats_by_directcost)
print("______________")
print("PROJECT LENGTH")

grouped = df_checking.groupby('startfiscalyear')
stats_by_directcost = grouped['project_length'].describe()
display(stats_by_directcost)

grouped = df_checking.groupby('allocation_niicode')
stats_by_directcost = grouped['project_length'].describe()
display(stats_by_directcost)


In [ ]:
pathname = "/Users/norika_machome/GitHub/IP_CAPSTONE"
df_limited_saimoku_institution.to_csv(pathname + "/Data/full_data/" + "df_c_a1" + ".csv", index = False)
print("Written to:", pathname + "/Data/full_data/" + "df_c_a1" + ".csv")

# Analysis 1: Quantity

In [ ]:
product = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/IP_csv/df_product.csv')
product.head(2)

In [ ]:
product_journal = product[(product["product_type"]=="journal_article") & (product["reviewed"]== True )]
product_journal.shape

In [ ]:
product_journal_count = product_journal
product_journal_count['journal_count'] = product_journal_count.groupby('awardnumber')['product_type'].transform('count')
product_journal_count = product_journal_count[['awardnumber','journal_count','year']]
#print(product_journal_count.awardnumber.describe()) #unique = 299836
product_journal_count = product_journal_count.groupby('awardnumber').first()
product_journal_count.head(5) #299836 rows × 2 columns

In [ ]:
#plt.scatter(x =product_journal_count.year, y = product_journal_count.journal_count )
#plt.title('all projects')


In [ ]:
df_all = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/df_c_a1.csv')

In [ ]:
df_all = df_all.set_index('awardnumber')
df_all_count = df_all.join(product_journal_count)
df_all_count["journal_count"] = df_all_count["journal_count"].fillna(0)
df_all_count.tail(2)


df_all_count.startfiscalyear = df_all_count.startfiscalyear.astype(int)
df_all_count = df_all_count.drop(['title_ja', 'title_en','institution_name', 'role','familyname', 'givenname','year','fullname'], axis=1)
df_all_count.head(10)

In [ ]:
df_all_count = df_all_count.reset_index()
df_all_count

In [ ]:

pathname = "/Users/norika_machome/GitHub/IP_CAPSTONE"
df_all_count.to_csv(pathname + "/Data/full_data/" + "c_quantity_0417" + ".csv", index = False)
print("Written to:", pathname + "/Data/full_data/" + "c_quantity_0417" + ".csv")

## Analysis 1: Quantity - Working paper

In [ ]:
product = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/IP_csv/df_product.csv')
product.head(2)

In [ ]:
product_journal = product[(product["product_type"]=="journal_article") & (product["reviewed"]!= True )]
product_journal.shape

In [ ]:
product_journal_count = product_journal
product_journal_count['journal_count'] = product_journal_count.groupby('awardnumber')['product_type'].transform('count')
product_journal_count = product_journal_count[['awardnumber','journal_count','year']]
print(product_journal_count.awardnumber.describe()) #unique = 384852


In [ ]:
product_journal_count = product_journal_count.groupby('awardnumber').first()
product_journal_count.head(5) #299836 rows × 2 columns


In [ ]:
df_all = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/df_c_a1.csv')
df_all = df_all.set_index('awardnumber')
df_all_count = df_all.join(product_journal_count)
df_all_count["journal_count"] = df_all_count["journal_count"].fillna(0)
df_all_count.tail(2)


df_all_count.startfiscalyear = df_all_count.startfiscalyear.astype(int)
df_all_count = df_all_count.drop(['title_ja', 'title_en','institution_name', 'role','familyname', 'givenname','year','fullname'], axis=1)
df_all_count.head(10)

df_all_count = df_all_count.reset_index()
df_all_count

In [ ]:
pathname = "/Users/norika_machome/GitHub/IP_CAPSTONE"
df_all_count.to_csv(pathname + "/Data/full_data/" + "c_quantity_0417_workingpaper" + ".csv", index = False)
print("Written to:", pathname + "/Data/full_data/" + "c_quantity_0417_workingpaper" + ".csv")

# Analysis 1: Quality

In [ ]:
product = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/IP_csv/df_product.csv')
product.head(2)
product_journal_doi= product[product.doi.notna()]
product_journal_doi = product_journal_doi[product_journal_doi['reviewed'] == True]
product_journal_doi

In [ ]:
product_journal_doi.sort_values(by = 'year')
#product_journal_doi.dtypes

In [ ]:
df = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/df_c_a1.csv')
#df['awardnumber'] = df['awardnumber'].astype(str)
#df.dtypes

In [ ]:
df = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/df_c_a1.csv')
df.head(2)


df_HS_sample = df[df['field_niicode'] == 'HS']
df_HS_sample = df_HS_sample.sample(frac = 0.20, random_state=123)

df_BIO_sample = df[df['field_niicode'] == 'BIO']
df_BIO_sample = df_BIO_sample.sample(frac = 0.20, random_state=123)

df_SE_sample = df[df['field_niicode'] == 'SE' ]
df_SE_sample = df_SE_sample.sample(frac = 0.20, random_state=123)

df_INT_sample = df[df['field_niicode'] == 'INT' ]
df_INT_sample = df_INT_sample.sample(frac = 0.20, random_state=123)

df_all_sample = pd.concat([df_HS_sample, df_BIO_sample, df_SE_sample, df_INT_sample], axis=0)


In [ ]:
df_all_sample.awardnumber.describe()
df_all_sample.head(1)
#product_journal_doi.head(1)

In [ ]:
df_final = pd.merge(df_all_sample, product_journal_doi, how = 'left',  on = 'awardnumber')
df_final.awardnumber.describe()

In [ ]:
df_final.startfiscalyear.value_counts()
df_final[df_final['startfiscalyear'] ==2013].head(30)

In [ ]:
df_final = df_final.drop(['title_ja',
       'title_en', 'institution_name', 'role', 'fullname', 'familyname', 'givenname',
       'index','product_type', 'sequence', 'reviewed', 'journaltitle_ja',
       'journaltitle_en', 'year'], axis=1)

In [ ]:
df_final.startfiscalyear.value_counts()

In [ ]:
df_all_quality_doi = df_final[df_final.doi.notna()] #3463
df_all_quality_doi_without = df_final[df_final.doi.isna()] #231 10
#
#pathname = "/Users/norika_machome/GitHub/IP_CAPSTONE"
#df_all_quality_doi.to_csv(pathname + "/Data/full_data/" + "c_quality_0417_sample" + ".csv", index = False)
#print("Written to:", pathname + "/Data/full_data/" + "b_quality_0417_sample" + ".csv")

In [ ]:
df_all_quality_doi.startfiscalyear.value_counts()

In [ ]:
df_all_quality_doi.startfiscalyear.value_counts()

In [ ]:
df_doi = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/c_quality_0912_doi.csv')
df_doi = df_doi.drop(['Volume', 'Issue', 'Beginning Page', 'Ending Page',
       'Article Number','Volume', 'Issue', 'Beginning Page', 'Ending Page',
       'Article Number','2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', '2022', '2023' ], axis=1)
df_doi.rename(columns={"DOI": "doi"}, inplace=True)
df_quality = df_quality.replace({556: 'INT', 682: 'SE',939: 'INT', 770: 'BIO',621:'HS'})

df_doi.head(4)

In [ ]:
df_quality_final = df_quality.merge(df_doi, how ='inner', on = 'doi')
df_quality_final.head()

In [ ]:
pathname = "/Users/norika_machome/GitHub/IP_CAPSTONE"
df_quality_final.to_csv(pathname + "/Data/full_data/" + "c_quality_0912_final" + ".csv", index = False)
print("Written to:", pathname + "/Data/full_data/" + "c_quality_0912_final" + ".csv")

# Category B

## Data Importing + Cleaning

SQL REQUIREMENTS:
- Duration: 2004-2017
- Category: B (niicode == 72)
- Section: 17 (general research)
- Project Status: closed (completed)
- Project Type: project

In [ ]:
sql = '''
SELECT *
FROM grantaward
WHERE grantaward.startfiscalyear BETWEEN 2004 AND 2017
AND grantaward.category_niicode = 69
AND grantaward.section_niicode = 17
AND grantaward.projectstatus_statuscode = "project_closed"
AND grantaward.projecttype = "project"
'''

# df : the original dataframe stated above
df = pd.read_sql_query(sql, engine)
df = df.set_index("awardnumber")

In [ ]:
df = df.drop(['projecttype','projectstatus_fiscalyear', 'projectstatus_statuscode','category_niicode',
       'section_niicode','institution_mextcode',
       'institution_jspscode', 'sequence', 'participate',
       'eradcode','familyname_yomi','givenname_yomi'], axis=1)
df['project_length'] = df['endfiscalyear'] - df['startfiscalyear'] + 1
df = df.replace({'hojokin': 0})
options = [3,4,5]
df = df[df["project_length"].isin(options)]
df = df.sort_values(by = 'startfiscalyear')
df.head(2)
#df.shape #(30166, 13)



In [ ]:
#pivot = pd.crosstab(df['startfiscalyear'], df['allocation_niicode'], margins = True)
#pivot

In [ ]:
sql_saimoku = '''
SELECT * 
FROM grantaward_field
'''
df_saimoku = pd.read_sql_query(sql_saimoku, engine)
df_saimoku.head()

In [ ]:
sql_institutiontype = '''
SELECT *
FROM master_institutions
'''
df_institution_type = pd.read_sql_query(sql_institutiontype, engine)
df_institution_type = df_institution_type[['institution_niicode', 'institution_table_group']]
df_institution_type


### Saimoku list
- 556 -> 939: Integraded Disciplines
- 621: Humanities and Social sciences
- 682: Sciences and Engineering
- 770 Biological Sciences

In [ ]:
options = [621, 682, 770, 556, 939]
df_saimoku_all = df_saimoku[df_saimoku['field_niicode'].isin(options)]
df_saimoku_all = df_saimoku_all.drop(['index', 'field_sequence', 'field_path','field_table','field_name'], axis=1)
df_saimoku_all.head() #325476 rows × 2 columns

In [ ]:
df = df.reset_index()
df.head(2)

In [ ]:
##### #df_saimoku_621 = df_saimoku_all[df_saimoku_all['field_niicode'] == 621]
##### #df_saimoku_682 = df_saimoku_all[df_saimoku_all['field_niicode'] == 682]
##### #df_saimoku_770 = df_saimoku_all[df_saimoku_all['field_niicode'] == 770]
##### #df_saimoku_556_939 = df_saimoku_all[(df_saimoku_all['field_niicode'] == 556) | (df_saimoku_all['field_niicode'] == 939 )]

In [ ]:
df_limited_saimoku = df.merge(df_saimoku_all,  how='inner', on = 'awardnumber')
df_limited_saimoku.head(2)

df_limited_saimoku = df_limited_saimoku.replace({556: 'INT', 682: 'SE',939: 'INT', 770: 'BIO',621:'HS'})
df_limited_saimoku.head(2)

In [ ]:
df_limited_saimoku_institution = df_limited_saimoku.merge(df_institution_type,  how='inner', on = 'institution_niicode')

In [ ]:
df_limited_saimoku_institution

In [ ]:
pathname = "/Users/norika_machome/GitHub/IP_CAPSTONE"
df_limited_saimoku_institution.to_csv(pathname + "/Data/full_data/" + "df_b_a2" + ".csv", index = False)
print("Written to:", pathname + "/Data/full_data/" + "df_b_a2" + ".csv")


# Analysis 2: Quantity

In [ ]:
product = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/IP_csv/df_product.csv')
product.head(2)

In [ ]:
product_journal = product[(product["product_type"]=="journal_article") & (product["reviewed"]== True )]
product_journal.head(2)

In [ ]:
product_journal_count = product_journal
product_journal_count['journal_count'] = product_journal_count.groupby('awardnumber')['product_type'].transform('count')
product_journal_count = product_journal_count[['awardnumber','journal_count','year']]
#print(product_journal_count.awardnumber.describe()) #unique = 299836
product_journal_count = product_journal_count.groupby('awardnumber').first()
product_journal_count.head(5) #299836 rows × 2 columns

In [ ]:
#plt.scatter(x =product_journal_count.year, y = product_journal_count.journal_count )
#plt.title('all projects')


In [ ]:
df_all = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/df_b_a2.csv')
df_all['awardnumber'] = df_all['awardnumber'].astype(str)


In [ ]:
df_all[df_all['startfiscalyear']==2004]

In [ ]:
df_all = df_all.set_index('awardnumber')
df_all_count = df_all.join(product_journal_count)
df_all_count["journal_count"] = df_all_count["journal_count"].fillna(0)
df_all_count.tail(2)


df_all_count.startfiscalyear = df_all_count.startfiscalyear.astype(int)
df_all_count = df_all_count.drop(['title_ja', 'title_en','institution_name', 'role','familyname', 'givenname','year','fullname'], axis=1)
df_all_count.head(20)

In [ ]:
df_all_count

In [ ]:
df_all_count = df_all_count.reset_index()

pathname = "/Users/norika_machome/GitHub/IP_CAPSTONE"
df_all_count.to_csv(pathname + "/Data/full_data/" + "b_quantity_0417" + ".csv", index = False)
print("Written to:", pathname + "/Data/full_data/" + "b_quantity_0417" + ".csv")

# Analysis 2: Quality

In [ ]:
df = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/df_b_a2.csv')
df.head(2)
df['awardnumber'] = df['awardnumber'].astype(str)
df.dtypes

In [ ]:

df_HS_sample = df[(df['startfiscalyear'] > 2009) & (df['field_niicode'] == 'HS') ]
df_HS_sample = df_HS_sample.sample(frac = 0.15, random_state=123)

df_BIO_sample = df[(df['startfiscalyear'] > 2009) &(df['field_niicode'] == 'BIO') ]
df_BIO_sample = df_BIO_sample.sample(frac = 0.15, random_state=123)

df_SE_sample = df[(df['startfiscalyear'] > 2009) &(df['field_niicode'] == 'SE') ]
df_SE_sample = df_SE_sample.sample(frac = 0.15, random_state=123)

df_INT_sample = df[(df['startfiscalyear'] > 2009) &(df['field_niicode'] == 'INT') ]
df_INT_sample = df_INT_sample.sample(frac = 0.15, random_state=123)

df_all_sample = pd.concat([df_HS_sample, df_BIO_sample, df_SE_sample, df_INT_sample], axis=0)


In [ ]:
pivot = pd.crosstab(df_all_sample['startfiscalyear'], df_all_sample['field_niicode'], margins = True)
pivot

In [ ]:
product = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/IP_csv/df_product.csv')
product.head(2)

product_journal = product[(product["product_type"]=="journal_article") & (product["reviewed"]== True ) ]
product_journal_doi= product_journal[product_journal.doi.notna()]

In [ ]:
product_journal_doi.head(2)
product_journal_doi.dtypes

In [ ]:
print((product_journal_doi['awardnumber'] == "23310056").any())


In [ ]:
df_final = pd.merge(df_all_sample, product_journal_doi, how = 'left',  on = 'awardnumber')
df_final = df_final.drop(['title_ja',
       'title_en', 'institution_name', 'role', 'fullname', 'familyname', 'givenname',
       'index','product_type', 'sequence', 'reviewed', 'journaltitle_ja',
       'journaltitle_en', 'year'], axis=1)
df_final

In [ ]:
df_all_quality_doi = df_final[df_final.doi.notna()] #3463
df_all_quality_doi_without = df_final[df_final.doi.isna()] #231 10

pathname = "/Users/norika_machome/GitHub/IP_CAPSTONE"
df_all_quality_doi.to_csv(pathname + "/Data/full_data/" + "b_quality_1011_sample" + ".csv", index = False)
print("Written to:", pathname + "/Data/full_data/" + "b_quality_1011_sample" + ".csv")

## Merging

In [ ]:
df_quality = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/b_quality_1011_sample.csv')
df_quality.head(2)

In [ ]:
df_doi = pd.read_csv('/Users/norika_machome/GitHub/IP_capstone/Data/full_data/b_quality_1011_doi.csv')
df_doi = df_doi.drop(['Volume', 'Issue', 'Beginning Page', 'Ending Page',
       'Article Number','Volume', 'Issue', 'Beginning Page', 'Ending Page',
       'Article Number','2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020', '2021', '2022', '2023' ], axis=1)
df_doi.rename(columns={"DOI": "doi"}, inplace=True)

df_doi.head(4)

In [ ]:
df_quality_final = df_quality.merge(df_doi, how ='inner', on = 'doi')
df_quality_final.head()

In [ ]:
pathname = "/Users/norika_machome/GitHub/IP_CAPSTONE"
df_quality_final.to_csv(pathname + "/Data/full_data/" + "b_quality_1011_final" + ".csv", index = False)
print("Written to:", pathname + "/Data/full_data/" + "b_quality_1011_final" + ".csv")